In [104]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import random
import yaml
import copy

### Initial population generation

In [87]:
# Run the Python script to generate initial population
!python generate_ensemble_yaml.py
# Import N, which is the population size (note: i could have done both running and importing by %run but doing so would be unclear how N was defined in this notebook so i took this two-step approach instead.
from generate_ensemble_yaml import N

!bash ensemble_simulation.sh

# list of file names for the population
files = ['result_%04i.nc' % (i + 1) for i in range(N)]
files = np.array(files)

INFO:root:Using template fabm.yaml...
INFO:root:  instances/1p1z/parameters/attCHL: 0.02486
INFO:root:  instances/1p1z/parameters/I_thNH4: 0.0095
INFO:root:  instances/1p1z/parameters/D_p5NH4: 0.1
INFO:root:  instances/1p1z/parameters/NitriR: 0.05
INFO:root:  instances/1p1z/parameters/K_NO3: 2.0
INFO:root:  instances/1p1z/parameters/K_NH4: 2.0
INFO:root:  instances/1p1z/parameters/Vp0: 0.69
INFO:root:  instances/1p1z/parameters/K_Phy: 0.5
INFO:root:  instances/1p1z/parameters/PhyIS: 0.025
INFO:root:  instances/1p1z/parameters/PhyMR: 0.072
INFO:root:  instances/1p1z/parameters/Chl2C_m: 0.0535
INFO:root:  instances/1p1z/parameters/ZooAE_N: 0.75
INFO:root:  instances/1p1z/parameters/ZooBM: 0.1
INFO:root:  instances/1p1z/parameters/ZooER: 0.1
INFO:root:  instances/1p1z/parameters/ZooGR: 0.75
INFO:root:  instances/1p1z/parameters/ZooMR: 0.025
INFO:root:  instances/1p1z/parameters/LDeRRN: 0.01
INFO:root:  instances/1p1z/parameters/SDeRRN: 0.03
INFO:root:  instances/1p1z/parameters/CoagR: 0.0

### fitness calculation
To do:

- consider rounding for disregarding subtle variations for correlations
- modify to accommodate the case when tarvars > 1

In [119]:
idx_time = range(100,366,10) # every 10 days after 100 days
idx_dep = range(1,110,10) # very 10 m from 1 m
tarvars = ["1p1z_no3"]#,"1p1z_chl"] # target variables

arr_fit = np.zeros(N) # array containing fitness

ds_tar = xr.open_dataset('result.nc') # control simulation results
for i in range(N): # loop over the population
    ds_ind = xr.open_dataset(files[i])
    for j in range(len(tarvars)):
        arr_fit[i] = xr.corr(ds_tar[tarvars[j]].isel(time=idx_time,z=idx_dep),ds_ind[tarvars[j]].isel(time=idx_time,z=idx_dep)).values
sorted_idx = np.argsort(arr_fit) # indices of the fitness array in the descending order (best fitness first, worst last)

# elitism (keep the best individul for the next generation)
source_file = files[sorted_idx][0]
dest_file = "result_best.nc"
!cp {source_file} {dest_file}

# discard the worst individual for the next generation
files_minus = files[sorted_idx[:-1]]

# random (

### Subsequent population generation
- elitism
- random
- crossover

In [159]:
prob_mutation = 0.1 # probability of mutation for each parameter offspring during crossover

# elitism: keep the best individult for the next generation
source_yaml = 'fabm_'+source_file[-7:-3]+'.yaml'
dest_yaml = 'fabm_0001_new.yaml' # 0001 is hard coded
!cp {source_yaml} {dest_yaml}

# random: for exploration of parameter space and prevention from local minima, generate a totally random individual
fabm = eatpy.models.gotm.YAMLFile("fabm.yaml")
for i in range(len(name_para)):
    fabm[parameter_path+name_para[i]] = np.round(random.uniform(l_bounds[i],u_bounds[i]),decimals=n_decimals[i])
# Open the new YAML file in write mode and save the modified content
with open('fabm_0002_new.yaml', 'w') as file: # 0002 is hard coded
    yaml.dump(fabm, file, default_flow_style=False)
if 1 == 0:
    # crossover
    for j in range(N-2): # generate N-2 individuals by crossover (the remaining two are generated via elitism and random)
        # Generate filename with zero padding (i+3 to start from 0003, as the first two are reserved for elitism and random
        file_new = f'fabm_{j+3:04}_next.yaml'    
        father = random.choice(files_minus)
        # Randomly select a mother, ensuring it is different from the father
        mother = random.choice([m for m in files_minus if m != father])
        father_yaml = 'fabm_'+father[-7:-3]+'.yaml'
        mother_yaml = 'fabm_'+mother[-7:-3]+'.yaml'
        # Open the YAML file and load its contents
        with open(father_yaml, 'r') as file:
            father_par = yaml.safe_load(file)
        with open(mother_yaml, 'r') as file:
            mother_par = yaml.safe_load(file)
        kid_par = copy.deepcopy(mother_par)
        for i in range(len(name_para)):
            # set the value of either parent
            kid_par[parameter_path+name_para[i]] = random.choice([father_par[parameter_path+name_para[i]],
                                                                  mother_par[parameter_path+name_para[i]]])
            # mutation with a specified probability: randomly select a value between the two parents
            if random.uniform(0,1) < prob_mutation:
                kid_par[parameter_path+name_para[i]] = np.round(random.uniform(father_par[parameter_path+name_para[i]],
                                                                           mother_par[parameter_path+name_para[i]]),
                                                            decimals=n_decimals[i])
        # Open the new YAML file in write mode and save the modified content
        with open(file_new, 'w') as file:
            yaml.dump(kid_par, file, default_flow_style=False)
        

In [134]:
father_par[parameter_path+name_para[i]]

KeyError: 'instances/1p1z/parameters/attCHL'

AttributeError: 'YAMLFile' object has no attribute 'dump'

In [152]:
fabm = eatpy.models.gotm.YAMLFile("fabm.yaml")
fabm[parameter_path+name_para[i]]

0.02486